In [0]:
# SEMMA

## SAMPLE

df = spark.table("analytics.olist.abt_ativacao").toPandas()
df.head(2)

In [0]:
df['dtRef'].unique()

In [0]:
# create OOT using the last moment of the data reference
df_oot = df[df['dtRef'] == df['dtRef'].max()].copy()

# train dataset
df_train = df[df['dtRef'] < df['dtRef'].max()].copy()

In [0]:
from sklearn import model_selection

features = df_train.columns[4:].tolist()
target = 'flagChurn'

X = df_train[features]
y = df_train[target]

# Teain and test split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    random_state=42, 
                                                                    test_size=0.2,
                                                                    stratify=y)

print("train target distribution: ", y_train.mean())
print(" test target distribution: ", y_test.mean())

In [0]:
## EXPLORE

# Check NA - count
train_isna = X_train.isna().sum()
train_isna[train_isna > 0]

In [0]:
# Check NA - mean
train_isna = X_train.isna().mean()
train_isna[train_isna > 0]

In [0]:
X_train['mediaAvaliacao'].describe()

In [0]:
X_train['mediaAvaliacao28d'].describe()

In [0]:
'mediaAvaliacao'
'mediaAvaliacao28d'
'mediaAvaliacao56d'
'mediaAvaliacao84d'
'mediaAvaliacao168d'
'mediaAvaliacao336d'
'pctNota1'
'pctNota2'
'pctNota3'
'pctNota4'
'pctNota5'
'pctNotaBaixa'
'pctTendencia1m_2m'
'pctTendencia1m_3m'
'pctTendencia1m_4m'
'pctTendencia1m_5m'
'pctTendencia1m_6m'
'pctTendencia1m_12m'
'qtdCliente'
'cliNovo'
'cliPontual'
'cliRecorrente'